In [1]:
VERSION = 'test'

%load_ext tensorboard

from model.model import *
from process_func import *
import tensorflow as tf
import os
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 
       'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles = ['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master']

# Run for validation

In [2]:
data = Titanic(DATA_DIR, show_head = False)
prepath = 'preprocessed'
try:
    if not os.path.exists(prepath):
        os.makedirs(prepath)
except OSError:
    print('Error Creating Directory...')
data.Preprocess(map, titles, VERSION)
data._data.to_csv(prepath + f"/encoded_{VERSION}.csv", index = False)
dataset = data.GetXandY()
# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [3]:
model = BinaryClassification(dataset['x_train'].shape)
CP_dir = SetCheckpoint(VERSION)
TB_dir = SetLog(VERSION)
# Run for validation

Model: "TitanicBC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 15)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                256       
                                                                 
 BN0 (BatchNormalization)    (None, 16)                64        
                                                                 
 Dropout0 (Dropout)          (None, 16)                0         
                                                                 
 Dense1 (Dense)              (None, 8)                 136       
                                                                 
 BN1 (BatchNormalization)    (None, 8)                 32        
                                                                 
 Dropout1 (Dropout)          (None, 8)                 0 

In [4]:
BATCH_SIZE = 128
EPOCHS = 300
callbacks = DefCallbacks(VERSION, CP = CP_dir, TB = TB_dir)
model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks, 
          validation_split = 0.3, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

Epoch 1/300
5/5 - 2s - loss: 1.2652 - val_loss: 0.7103 - 2s/epoch - 422ms/step
Epoch 2/300
5/5 - 0s - loss: 1.2581 - val_loss: 0.7034 - 92ms/epoch - 18ms/step
Epoch 3/300
5/5 - 0s - loss: 1.1264 - val_loss: 0.6965 - 93ms/epoch - 19ms/step
Epoch 4/300
5/5 - 0s - loss: 1.0426 - val_loss: 0.6901 - 98ms/epoch - 20ms/step
Epoch 5/300
5/5 - 0s - loss: 0.9045 - val_loss: 0.6838 - 109ms/epoch - 22ms/step
Epoch 6/300
5/5 - 0s - loss: 0.8722 - val_loss: 0.6782 - 95ms/epoch - 19ms/step
Epoch 7/300
5/5 - 0s - loss: 0.8520 - val_loss: 0.6727 - 88ms/epoch - 18ms/step
Epoch 8/300
5/5 - 0s - loss: 0.8267 - val_loss: 0.6675 - 86ms/epoch - 17ms/step
Epoch 9/300
5/5 - 0s - loss: 0.7524 - val_loss: 0.6629 - 90ms/epoch - 18ms/step
Epoch 10/300
5/5 - 0s - loss: 0.7022 - val_loss: 0.6590 - 87ms/epoch - 17ms/step
Epoch 11/300
5/5 - 0s - loss: 0.6943 - val_loss: 0.6554 - 95ms/epoch - 19ms/step
Epoch 12/300
5/5 - 0s - loss: 0.6744 - val_loss: 0.6516 - 96ms/epoch - 19ms/step
Epoch 13/300
5/5 - 0s - loss: 0.6196 

In [5]:
model.load_weights(CP_dir)
pred = model.predict(dataset['x_test'], batch_size=200)
pred = tf.reshape(pred, (-1))
data_check =  pd.read_csv(DATA_DIR + "gender_submission.csv")
pred_series = pd.Series(pred)
print(pred_series[:10])
# Creating new column of predictions in data_check dataframe
data_check['check'] = pred_series
series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
data_check['final'] = series
match = 0
nomatch = 0
for val in data_check.values:
    if val[1] == val[3]:
        match += 1
    else:
        nomatch += 1

print(f'Accuracy : {match/data_check.shape[-2] * 100 : .2f} %')
# Run for validation

3/3 [==============================] - 0s 2ms/step
0    0.130462
1    0.780625
2    0.130780
3    0.105491
4    0.910959
5    0.185417
6    0.726250
7    0.111542
8    0.620541
9    0.029178
dtype: float32
Accuracy :  93.06 %


In [6]:
temp = pd.DataFrame(pd.read_csv(DATA_DIR + "test.csv")['PassengerId'])
temp['Survived'] = data_check['final']
temp.to_csv(f"submission/submission_{VERSION}.csv", index = False)
# Run for validation

In [7]:
# Title (Mr. Miss. ) - Married?
# Social class from Ticket class and Fare